In [26]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import featuretools as ft
import lightgbm as lgb
from lightgbm import plot_tree
from graphviz import Digraph
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold,GroupKFold
from sklearn.metrics import roc_auc_score
import time
import pickle

%matplotlib inline

# Load Original Features

In [2]:
feat_num = 440

In [3]:
#df_total_raw = pd.read_csv('./data/features%s.csv'%(feat_num))
with open('./data/features%s.pickle'%(feat_num), 'rb') as handle:
    df_total_raw = pickle.load(handle)

In [4]:
#to_drop = ['TransactionDT']
#to_drop = ['V'+str(i) for i in range(1,340)]
with open('./data/feat%s_rm_pm_importance100.pickle'%(437), 'rb') as handle:
    to_drop = pickle.load(handle)

In [5]:
df_total = df_total_raw.drop(list(to_drop),axis=1)
#df_total = df_total_raw

In [6]:
features_train = df_total[df_total['isFraud'].notnull()]
features_test = df_total[df_total['isFraud'].isnull()]

In [7]:
labels_train = features_train['isFraud']
features_train = features_train.drop(columns = ['isFraud', 'TransactionID'])

In [8]:
features_train.shape

(590540, 338)

# Prepare model

In [9]:
categorical = ['ProductCD', 'card2', 'card3', 'card4', 'card5','card6',
              'addr1','addr2','P_email','R_email','M1','M2','M3',
              'M4','M5','M6','M7','M8','M9','DeviceType','DeviceInfo','dow','hour',
              'Device_name','Device_version','screen_width','screen_height',
               'P_email_suffix','R_email_suffix','id_30_OS','id_30_version',
              'is_card_freq_Device','is_wide','is_long','is_zero','is_win8_vista',
              'is_windows_otheros','is_card_freq_pdc','is_card_freq_addr1'] 
ids = [ 'id_%s'%(i) for i in range(12,39)]
categorical = categorical + ids

In [10]:
categorical = list(set(categorical).intersection(df_total.columns))

In [11]:
features_test_new = features_test.drop(columns = ['isFraud', 'TransactionID'])

In [22]:
params = {'objective':'binary',
          'boosting_type':'gbdt',
          'metric':'auc',
          'learning_rate':0.006,
          'num_leaves': 2**8,
          'max_depth':-1,
          'colsample_bytree': 0.7,# feature_fraction
          'subsample_freq':1,
          'subsample':0.7,
          'verbose':-1,
          'seed': 42#47,
                } 

In [21]:
params1 = {'objective': 'binary',# Core Parameters
          "boosting_type":"gbdt",#'goss' # Core Parameters
          "metric": 'auc',
          'learning_rate': 0.006883242363721497,
          'num_leaves': 491,# This is the main parameter to control the complexity of the tree model
          'max_depth': -1,# You also can use max_depth to limit the tree depth explicitly.
          'min_data_in_leaf': 106,# This is a very important parameter to prevent over-fitting in a leaf-wise tree. 
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,`a
          'feature_fraction_seed':11,
          'bagging_fraction': 0.4181193142567742,#randomly select part of data without resampling #control overfitting
          "bagging_seed": 11,
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,#L1
          'reg_lambda': 0.6485237330340494,#L2
          'random_state': 47
          #'num_threads':10
          #'device' :'gpu',
          #'is_unbalance':True
          #'scale_pos_weight':9
         }

# Feature Selection

In [14]:
features_train.tail()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,DeviceType,DeviceInfo,P_email,R_email,screen_width,pdc_amt_ratio,card1_fq_enc,addr1_fq_enc,card_mv_day_fq,card_DeviceType_fq
590535,15811047,49.00,0,6550,NaN,150.0,2,226.0,1,272.0,...,-1,-1,-1,-1,-1,0.318986,2110,35929.0,1,NaN
590536,15811049,39.50,0,10444,225.0,150.0,1,224.0,1,204.0,...,-1,-1,0,-1,-1,0.257142,25,77069.0,1,NaN
590537,15811079,30.95,0,12037,595.0,150.0,1,224.0,1,231.0,...,-1,-1,0,-1,-1,0.201482,1373,13532.0,2,NaN
590538,15811088,117.00,0,7826,481.0,150.0,1,224.0,1,387.0,...,-1,-1,7,-1,-1,0.761660,5021,15737.0,2,NaN
590539,15811131,279.95,0,15066,170.0,150.0,1,102.0,0,299.0,...,-1,-1,0,-1,-1,1.822452,14606,85045.0,2,NaN


In [ ]:
start = time.time()
train_set = lgb.Dataset(features_train.iloc[0:472432,:], label=labels_train.values[0:472432],categorical_feature=categorical)#
valid_set = lgb.Dataset(features_train.iloc[472432:,:], label=labels_train.values[472432:],categorical_feature=categorical)#
valid_results = {}
model = lgb.train(params,train_set,num_boost_round = 10000, 
                  valid_sets = [train_set, valid_set],
                  verbose_eval=500,
                  early_stopping_rounds = 500,
                  evals_result=valid_results)
print(time.time()-start)

In [ ]:
max(valid_results['valid_1']['auc'])

In [ ]:
lgb.plot_importance(model, max_num_features=128,figsize=(20,15))
#lgb.plot_split_value_histogram(model, feature='dist1', bins='auto')
lgb.plot_metric(valid_results, metric='auc')

In [ ]:
lgb.plot_tree(model,tree_index=0, figsize=(107,105))
graph = lgb.create_tree_digraph(model, tree_index=0, name='Tree0')
graph.render(view=True)

# Train Model

In [15]:
def fold_train_model(splits_num,features_train,labels_train,categorical):
    splits = splits_num
    folds = KFold(n_splits = splits,random_state=50)
    predictions = np.zeros(len(features_test_new))
    ave_auc = 0
    
    for fold_num, (trn_idx, val_idx) in enumerate(folds.split(features_train.values, 
                                                          labels_train.values)):
        print("Fold {}".format(fold_num))
        train_df, y_train_df = features_train.iloc[trn_idx], labels_train.iloc[trn_idx]
        valid_df, y_valid_df = features_train.iloc[val_idx], labels_train.iloc[val_idx]

        trn_data = lgb.Dataset(train_df, label=y_train_df,categorical_feature=categorical)
        val_data = lgb.Dataset(valid_df, label=y_valid_df,categorical_feature=categorical)

        valid_results = {}
        clf = lgb.train(params,
                        trn_data,
                        10000,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=500,
                        early_stopping_rounds=500,
                       evals_result=valid_results)

        pred = clf.predict(valid_df)
        auc_score = roc_auc_score(y_valid_df, pred)
        ave_auc += auc_score / splits
        predictions += clf.predict(features_test_new) / splits
    return ave_auc,predictions

In [24]:
ave_auc,predictions = fold_train_model(6,features_train,labels_train,categorical)

Fold 0


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.983332	valid_1's auc: 0.917856
[1000]	training's auc: 0.996289	valid_1's auc: 0.928191
[1500]	training's auc: 0.999139	valid_1's auc: 0.929697
[2000]	training's auc: 0.999808	valid_1's auc: 0.929903
[2500]	training's auc: 0.999962	valid_1's auc: 0.929734
Early stopping, best iteration is:
[2033]	training's auc: 0.999827	valid_1's auc: 0.930025
Fold 1


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.983523	valid_1's auc: 0.923405
[1000]	training's auc: 0.99647	valid_1's auc: 0.933734
[1500]	training's auc: 0.999215	valid_1's auc: 0.937031
[2000]	training's auc: 0.999834	valid_1's auc: 0.938451
[2500]	training's auc: 0.999971	valid_1's auc: 0.939103
[3000]	training's auc: 0.999996	valid_1's auc: 0.939086
Early stopping, best iteration is:
[2719]	training's auc: 0.999987	valid_1's auc: 0.939268
Fold 2


...

Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.984932	valid_1's auc: 0.929232
[1000]	training's auc: 0.997013	valid_1's auc: 0.934609
[1500]	training's auc: 0.999393	valid_1's auc: 0.934924
Early stopping, best iteration is:
[1302]	training's auc: 0.998853	valid_1's auc: 0.935097
Fold 4


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.983355	valid_1's auc: 0.936585
[1000]	training's auc: 0.99647	valid_1's auc: 0.947592
[1500]	training's auc: 0.999272	valid_1's auc: 0.949674
[2000]	training's auc: 0.999861	valid_1's auc: 0.950155
[2500]	training's auc: 0.999976	valid_1's auc: 0.950363
[3000]	training's auc: 0.999997	valid_1's auc: 0.950178
Early stopping, best iteration is:
[2618]	training's auc: 0.999985	valid_1's auc: 0.950395
Fold 5


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.984176	valid_1's auc: 0.922858
[1000]	training's auc: 0.996661	valid_1's auc: 0.93251
[1500]	training's auc: 0.999283	valid_1's auc: 0.934682
[2000]	training's auc: 0.999851	valid_1's auc: 0.935919
[2500]	training's auc: 0.999974	valid_1's auc: 0.936444
[3000]	training's auc: 0.999997	valid_1's auc: 0.936963
[3500]	training's auc: 1	valid_1's auc: 0.936925
Early stopping, best iteration is:
[3098]	training's auc: 0.999998	valid_1's auc: 0.93703


In [25]:
# feat440 new param 6 fold
ave_auc

0.9396926157221407

In [17]:
# feat440 new param 5 fold
ave_auc

0.9364543671827509

In [34]:
# feat440 add card_DeviceType_fq
ave_auc

0.9383068452162177

In [18]:
# feat439 na -999
ave_auc

0.9381576015559274

In [25]:
# feat439 num of leave 300
ave_auc

0.9368800258774106

In [16]:
# feat439 lr 0.005
ave_auc

0.9378605282673502

In [206]:
#feat439 6 fold
ave_auc

0.9423037793396379

In [174]:
# feat440 add id_31_fq_enc
ave_auc

0.9381709259349416

In [159]:
# feat440 add id_30_fq_enc
ave_auc

0.9380367451399003

In [128]:
# feat439 change device info
ave_auc

0.9380337132311257

In [112]:
# feat439 add card_mv_day_fq
ave_auc

0.9382558553226678

In [97]:
# feat441 drop card4_fraud_rate and pemail fraud rate
ave_auc

0.9378070966039451

In [73]:
# feat441 drop feat437_rm_pm_importance100 drop hour amt and card4_fraud_rate
ave_auc

0.9375985902362556

In [61]:
# feat441 drop feat437_rm_pm_importance100 drop hour amt
ave_auc

0.9375681374967423

In [46]:
# feat441 drop feat437_rm_pm_importance100 
ave_auc

0.9375681381106405

In [17]:
# feat438 add addr1 cnt drop feat437_rm_pm_importance100
ave_auc

0.9378057475398072

In [34]:
# feat437 add card1 cnt drop feat437_rm_pm_importance100
ave_auc

0.9372995256863785

In [17]:
# feat437 add card1 cnt drop transactionDT
ave_auc

0.9347757078114228

In [68]:
# feat437 add card1 cnt
ave_auc

0.9371691266159269

In [33]:
# feat436 add pdc amt ratio
ave_auc

0.9360835858343227

In [31]:
# feat457 drop Vfeature
ave_auc

0.9320555176749314

In [16]:
# feat 457 drop c8
ave_auc

0.9354828335082059

In [19]:
# feat 457 all features
ave_auc

0.9352800271384787

In [18]:
id_test = features_test['TransactionID']
submission = pd.DataFrame({'TransactionID': id_test, 
                            'isFraud': predictions})
submission.to_csv('./data/sub_feat%s_drop100_%s.csv'%(feat_num,'new_param'), index = False)
#submission.to_csv('./data/sub_feat%s_drop100.csv'%(feat_num), index = False)


# Recursive Test

In [53]:
features_train.iloc[:,-15:-1].head()

,pdc_amt_ratio,card1_fq_enc,addr1_fq_enc,card_mv_day_fq,card_DeviceType_fq,pdc_hour_Amt_mean,pdc_hour_Amt_ratio,pdc_month_Amt_mean,pdc_month_Amt_ratio,card_id_30_fq,card_id_31_fq,pdc_addr_Amt_ratio,addr_Amt_ratio,dev_Amt_ratio
0,0.445929,56,43035.0,1,NaN,248.961084,0.275143,151.395914,0.452456,NaN,NaN,0.499882,0.505295,NaN
1,0.188788,1338,76902.0,1,NaN,248.961084,0.116484,151.395914,0.191551,NaN,NaN,0.175652,0.182692,NaN
2,0.384085,1794,48387.0,1,NaN,248.961084,0.236985,151.395914,0.389707,NaN,NaN,0.393093,0.446788,NaN
3,0.325496,7635,17455.0,11,NaN,248.961084,0.200835,151.395914,0.330260,NaN,NaN,0.355220,0.363424,NaN
4,0.686003,30,7107.0,1,1.0,75.840000,0.659283,69.407541,0.720383,1.0,1.0,0.737484,0.299817,0.737964


In [56]:
to_drop = {'pdc_hour_Amt_mean','pdc_hour_Amt_ratio','pdc_amt_ratio',
          'pdc_month_Amt_mean','pdc_month_Amt_ratio','card_id_30_fq',
          'card_id_31_fq','pdc_addr_Amt_ratio','addr_Amt_ratio','dev_Amt_ratio'}

test = ['pdc_month_Amt_mean','pdc_month_Amt_ratio','pdc_addr_Amt_ratio','addr_Amt_ratio']
result = []
for col in test:
    print(col)
    to_drop_temp = list(to_drop - set([col]))
    features_train_temp = features_train.drop(to_drop_temp,axis=1)
    print(features_train_temp.shape)
    categorical_temp = list(set(categorical).intersection(features_train_temp.columns))
    ave_auc,predictions = fold_train_model(5,features_train_temp,labels_train,categorical_temp)
    print(ave_auc)
    id_test = features_test['TransactionID']
    submission = pd.DataFrame({'TransactionID': id_test, 
                            'isFraud': predictions})
    submission.to_csv('./data/sub_feat%s_drop100_%s_test.csv'%(feat_num,col), index = False)
    result.append([col,ave_auc])

pdc_month_Amt_mean
(590540, 339)
Fold 0


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.986556	valid_1's auc: 0.911225
[1000]	training's auc: 0.997586	valid_1's auc: 0.922818
[1500]	training's auc: 0.999632	valid_1's auc: 0.925309
[2000]	training's auc: 0.999957	valid_1's auc: 0.925782
[2500]	training's auc: 0.999997	valid_1's auc: 0.926027
[3000]	training's auc: 1	valid_1's auc: 0.926112
[3500]	training's auc: 1	valid_1's auc: 0.92637
[4000]	training's auc: 1	valid_1's auc: 0.926608
Early stopping, best iteration is:
[3904]	training's auc: 1	valid_1's auc: 0.926569
Fold 1


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.987175	valid_1's auc: 0.932536
[1000]	training's auc: 0.99798	valid_1's auc: 0.940376
[1500]	training's auc: 0.999736	valid_1's auc: 0.940994
Early stopping, best iteration is:
[1439]	training's auc: 0.999658	valid_1's auc: 0.941049
Fold 2


...

/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.986793	valid_1's auc: 0.910565
[1000]	training's auc: 0.997702	valid_1's auc: 0.922389
[1500]	training's auc: 0.999669	valid_1's auc: 0.924595
[2000]	training's auc: 0.999962	valid_1's auc: 0.925192
[2500]	training's auc: 0.999997	valid_1's auc: 0.92574
[3000]	training's auc: 1	valid_1's auc: 0.926145
[3500]	training's auc: 1	valid_1's auc: 0.926205
[4000]	training's auc: 1	valid_1's auc: 0.926385
Early stopping, best iteration is:
[3930]	training's auc: 1	valid_1's auc: 0.926402
Fold 1


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.987376	valid_1's auc: 0.931649
[1000]	training's auc: 0.998111	valid_1's auc: 0.939517
[1500]	training's auc: 0.999766	valid_1's auc: 0.940091


KeyboardInterrupt: 

In [52]:
result

[['pdc_hour_Amt_mean', 0.9380002088591793],
 ['pdc_month_Amt_ratio', 0.9380738534486257],
 ['addr_Amt_ratio', 0.9381503901533759],
 ['pdc_hour_Amt_ratio', 0.9378806075783767],
 ['pdc_addr_Amt_ratio', 0.9381000559605122],
 ['dev_Amt_ratio', 0.9383520940351144],
 ['card_id_31_fq', 0.9380337093537161],
 ['card_id_30_fq', 0.9380861626171006],
 ['pdc_month_Amt_mean', 0.9384083713725051]]

In [17]:
result

[['card_DeviceInfo_fq', 0.938143843412518],
 ['card_DeviceType_fq', 0.9383068452162177],
 ['card_screen_width_fq', 0.9381135438426036]]

In [191]:
#'DT_hour_Amt_ratio','DT_day_Amt_ratio','DT_month_Amt_ratio',
#           'DT_year_Amt_ratio','card2_Amt_ratio',
#          'card3_Amt_ratio','card4_Amt_ratio','card5_Amt_ratio','card6_Amt_ratio'
result

[['card5_Amt_ratio', 0.9378145865335745],
 ['DT_day_Amt_ratio', 0.9378640126104663],
 ['card4_Amt_ratio', 0.937709725015683],
 ['card2_Amt_ratio', 0.937458223863788],
 ['DT_month_Amt_ratio', 0.9377889434647904],
 ['card6_Amt_ratio', 0.937863676290689],
 ['DT_hour_Amt_ratio', 0.9377024436406989],
 ['card3_Amt_ratio', 0.9376937326146639],
 ['DT_year_Amt_ratio', 0.9379433974493159]]